In [1]:
# Importing libraries
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd 
import joblib

In [2]:
# Defining the path of model
model = joblib.load(r"C:\Users\vinay\Downloads\fraud_detection_model.joblib")
model

c:\Users\vinay\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\vinay\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


AttributeError: 'RandomForestClassifier' object has no attribute 'monotonic_cst'

AttributeError: 'RandomForestClassifier' object has no attribute 'monotonic_cst'

AttributeError: 'RandomForestClassifier' object has no attribute 'monotonic_cst'

In [3]:

# Read dataset
new_data =  pd.read_csv(r"C:\Users\vinay\Downloads\creditcard.csv")

# Converting date column to datetime and seeing few rows and columns

new_data['Time'] = pd.to_datetime(new_data['Time'])
new_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1970-01-01 00:00:00.000000000,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1970-01-01 00:00:00.000000000,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1970-01-01 00:00:00.000000001,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1970-01-01 00:00:00.000000001,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,1970-01-01 00:00:00.000000002,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Feature Engineering
# Time features
new_data['Hour'] = new_data['Time'].dt.hour
new_data['Day'] = new_data['Time'].dt.day
new_data['Month'] = new_data['Time'].dt.month

# Amount-based features
new_data['Amount_Bin'] = pd.qcut(new_data['Amount'], q=5, labels=False)

# One-hot encoding for categorical variables
cat_cols = new_data.select_dtypes(include=['object']).columns
new_data = pd.get_dummies(new_data, columns=cat_cols)

In [5]:
# Store the Feature Matrix in X and Target in Vector y
X = new_data.drop(['Class', 'Time'], axis=1)
y = new_data['Class']

# Split the dataset into Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:

# Define the numeric and categorical features
numeric_features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
categorical_features = ['Hour', 'Day', 'Month', 'Amount_Bin']

# Get the column names after one-hot encoding
columns = X_train.columns

# Define the preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [7]:

# Combine preprocessing and feature engineering steps
full_processor = FeatureUnion([
    ('preprocessor', preprocessor)
])

# Create the pipeline
fraud_detection_pipeline = Pipeline(steps=[
    ('processor', full_processor),
    ('classifier', RandomForestClassifier())
])

In [8]:

# Train the pipeline
fraud_detection_pipeline.fit(X_train, y_train)

Pipeline(steps=[('processor',
                 FeatureUnion(transformer_list=[('preprocessor',
                                                 ColumnTransformer(transformers=[('num',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer()),
                                                                                                  ('scaler',
                                                                                                   StandardScaler())]),
                                                                                  ['V1',
                                                                                   'V2',
                                                                                   'V3',
                                                                                   'V4',
                                                                                   'V5',
                                                                                   'V6',
                                                                                   'V7',
                                                                                   'V8',
                                                                                   'V9',
                                                                                   'V10',
                                                                                   'V11',
                                                                                   'V12',
                                                                                   'V13',
                                                                                   'V14',
                                                                                   'V15',
                                                                                   'V16',
                                                                                   'V17',
                                                                                   'V18',
                                                                                   'V19',
                                                                                   'V20',
                                                                                   'V21',
                                                                                   'V22',
                                                                                   'V23',
                                                                                   'V24',
                                                                                   'V25',
                                                                                   'V26',
                                                                                   'V27',
                                                                                   'V28',
                                                                                   'Amount']),
                                                                                 ('cat',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('one_hot',
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['Hour',
                                                                       

In [9]:
# Make predictions
y_pred = fraud_detection_pipeline.predict(X_test)

# Save the pipeline
import joblib
joblib.dump(fraud_detection_pipeline, 'fraud_detection_pipeline.joblib')

['fraud_detection_pipeline.joblib']